In [1]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz (12.0 MB)
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019128 sha256=87de7355d67338eb566661da108359a18551401e3b7e53dab110692200fc44df
  Stored in directory: c:\users\131b~1\appdata\local\pip\cache\wheels\64\69\41\6f820cf1d7488a0381a2059f66ec9f8f23116f7c67d18f3d8d
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.0.0
    Uninstalling en-core-web-sm-2.0.0:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\en_core_web_sm-2.0.0.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.



In [18]:
import spacy
!python -m spacy download en
from spacy.lang.en import English
parser = English()
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074438 sha256=139055437e23d4682ad9a59bce319c3d885857ebf0051fb1e8211157deb00e82
  Stored in directory: C:\Users\131B~1\AppData\Local\Temp\pip-ephem-wheel-cache-cxprrwj6\wheels\59\4f\8c\0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.0.0
    Uninstalling en-core-web-sm-2.0.0:


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\en_core_web_sm-2.0.0.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.



error: bad escape \p at position 257

In [19]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [20]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\רועי\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [22]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [23]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\רועי\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [25]:
import random
text_data = []
with open('Topic_modeling_Example.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['semantic', 'application', 'using', 'information', 'framework']
['route', 'performance', 'dependent', 'costs']
['noise', 'reduction', 'system', 'base', 'system', 'identification', 'variable']
['reconfigurable', 'array', 'concurrent', 'support', 'multiple', 'radio', 'standard', 'flexible', 'mapping']
['online', 'dynamic', 'reordering', 'interactive', 'processing']
['exploration', 'science', 'instrument', 'observatory']
['leveraging', 'interlingual', 'classification', 'improve', 'search']
['robot', 'rockstars', 'mix', 'reality']
['analysis', 'blocking', 'probability', 'noise', 'crosstalk', 'impair', 'optical', 'network']
['spectrum', 'cloud', 'session', 'base', 'spectrum', 'trading', 'system', 'multi', 'cognitive', 'radio', 'network']
['division', 'parallel', 'delta', 'sigma', 'converter', 'probabilistic', 'iterative', 'decoding']
['modular', 'reduction', 'without', 'precomputational', 'phase']
['analog', 'implementation', 'base', 'adaptation', 'denoising']
['panoramic', 'imaging', 'sys

In [26]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [27]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [28]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

### Try 5 topics

In [29]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [30]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.030*"sigma" + 0.030*"delta" + 0.030*"modulator" + 0.016*"dynamic"')
(1, '0.054*"system" + 0.028*"base" + 0.028*"spectrum" + 0.019*"iterative"')
(2, '0.018*"reduction" + 0.018*"approach" + 0.018*"building" + 0.018*"compositional"')
(3, '0.026*"network" + 0.026*"processing" + 0.014*"noise" + 0.014*"comparative"')
(4, '0.046*"system" + 0.031*"flexible" + 0.017*"image" + 0.017*"design"')


In [31]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(179, 1)]
[(0, 0.100036986), (1, 0.59985054), (2, 0.100040704), (3, 0.10003277), (4, 0.100039005)]


In [32]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.021*"spectrum" + 0.014*"base" + 0.014*"sigma" + 0.014*"delta"')
(1, '0.063*"system" + 0.020*"reduction" + 0.011*"base" + 0.011*"noise"')
(2, '0.018*"flexible" + 0.010*"processing" + 0.010*"comparative" + 0.010*"radio"')


In [33]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.033*"control" + 0.033*"application" + 0.033*"balancing" + 0.033*"cycle"')
(1, '0.097*"system" + 0.059*"base" + 0.040*"spectrum" + 0.040*"network"')
(2, '0.031*"deterministic" + 0.031*"modelling" + 0.031*"update" + 0.031*"deductive"')
(3, '0.023*"flexible" + 0.023*"bandwidth" + 0.023*"dynamic" + 0.023*"modulator"')
(4, '0.033*"reduction" + 0.033*"comparative" + 0.033*"phase" + 0.033*"page"')
(5, '0.029*"processing" + 0.029*"study" + 0.029*"statistical" + 0.029*"cell"')
(6, '0.023*"image" + 0.023*"fourth" + 0.023*"distortion" + 0.023*"order"')
(7, '0.026*"sigma" + 0.026*"delta" + 0.026*"incremental" + 0.026*"community"')
(8, '0.029*"design" + 0.029*"iterative" + 0.029*"decoding" + 0.029*"efficient"')
(9, '0.028*"noise" + 0.028*"optical" + 0.028*"mismatch" + 0.028*"modulators"')


### pyLDAvis

In [34]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [35]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [36]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)